In [1]:
import pandas as pd
import cleaningtools as ct 
import file_config as fconfig
import merge_config as config
import numpy as np
import math


check for pandas


In [2]:
def clean_for_merge(data):
    for col in data.columns:
        if 'depth_ft'in col.lower():
            data['from_ft']=data[col]
            data['to_ft']=data['from_ft']+9
            #convert ft to meters
            data['from_m']=data['from_ft']* .3281
            data['to_m']=data['to_ft'] * 3.281
            data=data.drop(data.filter(like='Depth').columns,axis=1)
        if ('hole' in col.lower())& ('id' in col.lower()):
            data.rename(columns={f'{col}':f'hole_id'},inplace=True)
        try: 
            data[col]=data[col].str.strip()
        except Exception as e:
            pass
            print(f'{col}: {e}')
        data.rename(columns={f'{col}':f'{col.strip().lower()}'},inplace=True)
        data=data.drop(data.filter(like='unnamed'),axis=1)
        if 'from_ft'in col.lower():
            try:
                drop_index=data[data['from_ft'].isna()].index
                data=data.drop(drop_index,axis=0)
            except Exception as e:
                print(f'{e}')
    if 'hole_id' not in data.columns:
        data['hole_id']=''
    names=data.loc[:,data.columns.duplicated()].columns
    if len(names)>0:
        print(f'duplcated column: {names} drop or else the nasty merge bug')
        data=data.loc[:,~data.columns.duplicated()].copy()

    return data

In [3]:
def merge_concat_onid(big_df,data,merge_col=['geo','sample_id','hole_id','from_ft','to_ft']):
    
    num_common=big_df[big_df.sample_id.isin(data.sample_id)].shape[0]
    print(f'{num_common} common ids, concat')

    
    if num_common==0:
        print(f'{num_common} common ids, concat')
        big_df=pd.concat([big_df,data],axis=0)
    else: 
        print (f'{num_common} common ids, merge')
        big_df=big_df.merge(data,on=merge_col,how='left')
    n_col=big_df.shape[1]
    print('duplicate columns: ', big_df.columns.duplicated().sum())
    print('duplicate indexs: ', big_df.index.duplicated().sum())
    print(f'{n_col} columns')
    return big_df.reset_index(drop=True)

In [5]:
big_df=pd.DataFrame()
data_list=[]
prev_cols=[]
for file in config.merge_samples:
    print(f"####### merge: {file.split('/')[-1]} #######")
    data=pd.read_csv(file)
    data=clean_for_merge(data)
    data=data.set_index('sample_id')
    data_list.append(data)
    cols=['hole_id', 'sample_id']
    [prev_cols.remove(col) for col in prev_cols if col not in data.columns]
    [prev_cols.append(col) for col in data.columns]
    dups=sorted(set([x for x in prev_cols if prev_cols.count(x) > 1]))
    print(f'common columns {dups}')

    big_df=pd.concat([big_df,data],axis=0,join='outer')
    print(f'data shape {big_df.shape}')
cols=config.main_columns
not_dups=[i for i in big_df.columns if i not in cols]
big_df=pd.concat([big_df[cols],big_df[not_dups]],axis=1)
    

####### merge: drill assays master.csv #######
common columns []
data shape (1299, 45)
####### merge: spectral master.csv #######
duplcated column: Index(['spectrumid'], dtype='object') drop or else the nasty merge bug
common columns ['end_depth', 'file', 'folder', 'hole_id', 'recvd wt.', 'start_depth']
data shape (2365, 161)
####### merge: hyp-package samples master.csv #######
common columns ['hole_id']
data shape (3445, 168)
####### merge: samples master.csv #######
common columns ['filename', 'foldername', 'from_ft', 'from_m', 'geo', 'hole_id', 'to_ft', 'to_m']
data shape (4759, 172)


In [6]:
big_df

,geo,hole_id,from_ft,to_ft,recvd wt.,au,au_2,ag,al,as,...,fintfeoxide,amph_vnir,foldername,filename,from_m,to_m,qaqc,description,_2,_3
sample_id,,,,,,,,,,,,,,,,,,,,,
B300001,NaN,,NaN,NaN,2.14,0.026,NaN,1.0,3.69,160.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B300002,NaN,,NaN,NaN,6.22,0.296,NaN,3.0,4.56,3670.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B300003,NaN,,NaN,NaN,5.05,0.040,NaN,1.0,4.51,970.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B300004,NaN,,NaN,NaN,4.22,0.028,NaN,1.0,3.71,340.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B300005,NaN,,NaN,NaN,4.82,0.133,NaN,4.0,2.39,1400.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C523556,RH,KM-22-30,869.0,874.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,KM-22-30,Kay Drill Log KM-21-30.xlsx,264.87,266.40,NaN,"silicified schist, 2-3% pyrite, 2% chalcopyrit...",556.0,C523556
C523557,RH,KM-22-30,874.0,879.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,KM-22-30,Kay Drill Log KM-21-30.xlsx,266.40,267.92,NaN,"silicified schist, 1-2% pyrite, 0.1% chalcopyrite",557.0,C523557
C523558,RH,KM-22-30,879.0,883.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,KM-22-30,Kay Drill Log KM-21-30.xlsx,267.92,269.14,NaN,"light gray schist, 1-2% pyrite, 0.1% chalcopyrite",558.0,C523558


In [ ]:
spec.loc[:,~spec.columns.duplicated()]

In [ ]:
data_list[2].head()

In [ ]:
data_list[3].head()

In [ ]:
list(big_df.columns)

In [ ]:
big_df.columns

In [ ]:
file=fconfig.spec_file
spec=pd.read_csv(file)
spec=clean_for_merge(spec)

In [ ]:
spec.sample_id.duplicated().sum()


In [ ]:
zipped=zip(data['sample_id'], data['From_ft'], data['To_Ft'])
df=pd.DataFrame([(id, y) for id, start, end in zipped for y in np.arange(start, end,.5)],
                   columns=['sample_id','depth'])
df